# Sesión 12: Regresión Lineal Múltiple

### Importando los paquetes necesarios

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline


<h2 id="understanding_data">Entendiendo la data</h2>

### `FuelConsumption.csv`:
Hemos descargado un conjunto de datos de consumo de combustible, **`FuelConsumption.csv`**, que contiene clasificaciones de consumo de combustible específicas del modelo y emisiones estimadas de dióxido de carbono para nuevos vehículos ligeros para la venta al por menor en Canadá. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** e.g. 2014
- **MAKE** e.g. Acura
- **MODEL** e.g. ILX
- **VEHICLE CLASS** e.g. SUV
- **ENGINE SIZE** e.g. 4.7
- **CYLINDERS** e.g 6
- **TRANSMISSION** e.g. A6
- **FUELTYPE** e.g. z
- **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
- **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0


<h2 id="reading_data">Leyendo los datos de entreada</h2>

In [ ]:
df = pd.read_csv("data/FuelConsumption.csv")

# take a look at the dataset
df.head()

Vamos a seleccionar algunas características que queremos usar para la regresión:

In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head(9)

Vamos a graficar los valores de Emission respecto a los de Engine size:

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### Creando el conjunto de datos de entrenamiento y prueba
La división Train/Test Split implica dividir el conjunto de datos en conjuntos de entrenamiento y prueba, respectivamente, que son mutuamente excluyentes. Después de lo cual, entrena con el conjunto Train y prueba con el conjunto Test.
Esto proporcionará una evaluación más precisa de la precisión fuera de la muestra porque el conjunto de datos de prueba no es parte del conjunto de datos que se ha utilizado para entrenar los datos. Es más realista para problemas del mundo real.

Esto significa que conocemos el resultado de cada punto de datos en este conjunto de datos, ¡por lo que es genial probarlo! Y dado que estos datos no se han utilizado para entrenar el modelo, el modelo no tiene conocimiento del resultado de estos puntos de datos. Entonces, en esencia, es realmente una prueba fuera de muestra.



In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

#### Distribución de la data de entrenamiento

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

<h2 id="multiple_regression_model">Modelo de Regresión Múltiple</h2>


En realidad, hay múltiples variables que predicen la emisión de Co2. Cuando hay más de una variable independiente presente, el proceso se llama regresión lineal múltiple. Por ejemplo, prediciendo la emisión de CO2 utilizando FUELCONSUMPTION_COMB, EngineSize y Cylinders de automóviles. Lo bueno aquí es que la regresión lineal múltiple es la extensión del modelo de regresión lineal simple.

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
y = np.asanyarray(train[['CO2EMISSIONS']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

CO2EMISSIONS = 66.99718234 + 11.5258905 * ENGINESIZE + 6.84610122 * CYLINDERS + 9.5820981 * FUELCONSUMPTION_COMB

Como se mencionó anteriormente, __Coefficient__ and __Intercept__ , son los parámetros de la línea de ajuste. 
Dado que es una regresión lineal múltiple, con 3 parámetros, y sabiendo que los parámetros son la intersección y los coeficientes del hiperplano, <i>sklearn</i> puede estimarlos a partir de nuestros datos. <i>Scikit-learn</i> utiliza el método de mínimos cuadrados ordinarios para resolver este problema.

#### Mínimos Cuadrados Ordinarios/Ordinary Least Squares (OLS)
OLS es un método para estimar los parámetros desconocidos en un modelo de regresión lineal. OLS elige los parámetros de una función lineal de un conjunto de variables explicativas minimizando la suma de los cuadrados de las diferencias entre la variable <i>target</i> dependiente y las predichas por la función lineal. En otras palabras, intenta minimizar la suma de los errores al cuadrado (SSE) o el error al cuadrado medio (MSE) entre la variable objetivo <i>(y)</i> y nuestra salida predicha ($\hat{y}$) sobre todas las muestras en el conjunto de datos.

OLS puede encontrar los mejores parámetros utilizando los siguientes métodos:
   - Resolviendo los parámetros del modelo analíticamente usando ecuaciones de forma cerrada.
   - Usando de un algoritmo de optimización (Descenso de gradiente, Descenso de gradiente estocástico, Método de Newton, etc.)

<h2 id="prediction">Predicción</h2>

In [ ]:
y_hat= regr.predict(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
x = np.asanyarray(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
y = np.asanyarray(test[['CO2EMISSIONS']])
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat - y)))
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('R2-score: %.2f' % regr.score(x, y))